In [1]:
# Slow but working solution, see below for second solution

import copy
from itertools import product, combinations

def generate_all_elements_in_cycle(cycle, n, m):
    """Generate all elements from a cycle."""
    flag = True
    while flag:
        flag = False
        for el_one in cycle:
            for el_two in cycle:
                res = [el_one[0] + el_two[0], el_one[1] + el_two[1]]
                if res[0] >= n:
                    res[0] -= n
                if res[1] >= m:
                    res[1] -= m
                if res not in cycle:
                    cycle.append(res)
                    flag = True
                    break

    cycle = list(cycle)
    cycle.sort()
    return cycle
        

def my_combinations(n, m):
    pairs = [list(x) for x in product(range(n), range(m))]
    for i in range(1, n * m):
        for r in combinations(pairs, i):
            yield r
    
def get_Zn_x_Zm_cycles(n, m):
    """
    Generate all combinations of elements that 'cycle' for Z^n
    
    Ex for n=1:
        for Z4 0 and 2 'cycle' : 0 -> 2 -> 0 -> ...
        for Z9 0 and 3 'cycle' : 0 -> 3 -> 6 -> 0 -> ...
    """
    results = []

    for pair in my_combinations(n, m):
        cycle = generate_all_elements_in_cycle(list(pair), n, m)
        if cycle not in results:
            results.append(cycle)

    return sorted(results, key=lambda x: len(x))

def pretty_print_cycles(n, m):
    result = get_Zn_x_Zm_cycles(n, m)
    for index, group in enumerate(result):
        print('\ngroup {}'.format(index + 1) + ' - ' * 10)
        print('[ ' + ', '.join(['({}, {})'.format(str(element[0]), str(element[1])) for element in group]) + ' ]')

In [2]:
# pretty_print_cycles(6, 6) # Works hard for n or m > 5 
pretty_print_cycles(2, 4) 


group 1 -  -  -  -  -  -  -  -  -  - 
[ (0, 0) ]

group 2 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 2) ]

group 3 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (1, 0) ]

group 4 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (1, 2) ]

group 5 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 1), (0, 2), (0, 3) ]

group 6 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 2), (1, 1), (1, 3) ]

group 7 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 2), (1, 0), (1, 2) ]

group 8 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3) ]


In [3]:
# Fast solution using divisor, I'm missing some subgroups, see below
from itertools import product


def generate_all_elements_in_cycle(cycle, n):
    """Generate all elements from a cycle."""
    flag = True
    while flag:
        flag = False
        for el_one in cycle:
            for el_two in cycle:
                res = el_one + el_two
                if res >= n:
                    res -= n
                if res not in cycle:
                    cycle.append(res)
                    flag = True
                    break

    cycle = list(cycle)
    cycle.sort()
    return cycle


def is_prime(n):
    """Custom is_prime to consider 0 and 1 as primes"""
    if (n <= 3): 
        return True 
    if (n % 2 == 0 or n % 3 == 0) : 
        return False
  
    i = 5
    while(i * i <= n): 
        if (n % i == 0 or n % (i + 2) == 0): 
            return False
        i = i + 6
  
    return True

    

def generate_cyclic_groups(n):
    """Generate all cyclic subgroups in Zn using divisors"""
    subgroups = []
    
    for i in range(n):
        if is_prime(i) is False:
            continue
            
        cycle = generate_all_elements_in_cycle([0, i], n)
        if cycle not in subgroups:
            subgroups.append(cycle)
    
    return subgroups


def get_Zn_x_Zm_cycles(n, m):
    n_groups = generate_cyclic_groups(n)
    m_groups = generate_cyclic_groups(m)

    results = []
    for i in product(n_groups, m_groups):
        # The issue is here, I'm missing a few list combinations so
        # not all are generated
        # From the given example I'm missing H4 and H7
        res = list(set(product(i[0], i[1])))
        
        res.sort()
        if res not in results:
            results.append(res)

    results.sort(key=lambda x: len(x))
    return results


def pretty_print_cycles(n, m):
    result = get_Zn_x_Zm_cycles(n, m)
    for index, group in enumerate(result):
        print('\ngroup {}'.format(index + 1) + ' - ' * 10)
        print('[ ' + ', '.join(['({}, {})'.format(str(element[0]), str(element[1])) for element in group]) + ' ]')


In [4]:
pretty_print_cycles(2, 4)


group 1 -  -  -  -  -  -  -  -  -  - 
[ (0, 0) ]

group 2 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 2) ]

group 3 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (1, 0) ]

group 4 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 1), (0, 2), (0, 3) ]

group 5 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 2), (1, 0), (1, 2) ]

group 6 -  -  -  -  -  -  -  -  -  - 
[ (0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3) ]
